# Evaluate Baselines

The goal of this notebook is to demonstrate how we can evaluate the results of a baseline on a given benchmark.

It will be split into two part. The first part will focus on the evaluation of a baseline that does not requires any training (the `DCApproximatrionAC`). On the second part, we will show how to load a baseline (or any other `AugmentedSimulator`) and evaluate it on a `Benchmark` of our choice.

As for the first notebook, we demonstrate this capability for the case of `NeuripsBenchmark1`.

**NB** This notebook supposes that the data for the benchmark are already available. If they are not, please generate them or download them.

**NB** The `DCApproximatrionAC` baseline requires the `grid2op` python package.

#### Import required packages

In [1]:
import pathlib
from pprint import pprint
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

In [2]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

#  Benchmark1

## Initial step: load the dataset

A common dataset will be used for evaluate the two augmented simulator. This initial step aims at loading it once and for all.

In [3]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [4]:
# to verify the config is loaded appropriately for this benchmark
print("Benchmark name: ", benchmark1.config.section_name)
print("Environment name: ", benchmark1.config.get_option("env_name"))
print("Output attributes: ", benchmark1.config.get_option("attr_y"))
print("Evaluation criteria: ")
pprint(benchmark1.config.get_option("eval_dict"))

Benchmark name:  Benchmark1
Environment name:  l2rpn_case14_sandbox
Output attributes:  ('a_or', 'a_ex')
Evaluation criteria: 
{'IndRed': ['TIME_INF'],
 'ML': ['MSE_avg', 'MAE_avg', 'mape_90_avg', 'TIME_INF'],
 'OOD': ['MSE_avg', 'MAE_avg', 'mape_90_avg', 'TIME_INF'],
 'Physics': ['CURRENT_POS']}


## The DC approximation

We remind that the `grid2op` library is required for this part. You can install it with `pip install grid2op` if you do not have it already.

First we will create the "augmented simulator". As opposed to the second model we will expose here, this method require access to a powergrid. This is one of the reason we need grid2op. 

The way to load each `AugmentedSimulator` is specific. Here for example we load the DCApproximation that will use the same powergrid as the one used to generate the data in the previous Notebook.

In [5]:
# the next few lines are specific for each benchmark and each `AugmentedSimulator`
import grid2op
import warnings
from lips.physical_simulator.dcApproximationAS import DCApproximationAS
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    env = grid2op.make(benchmark1.config.get_option("env_name"), test=True)
    grid_path = pathlib.Path(env.get_path_env()) / "grid.json"

dc_sim = DCApproximationAS(name="dc_approximation", 
                           benchmark_name="Benchmark1",
                           config_path=BENCH_CONFIG_PATH,
                           grid_path=grid_path)

Now that the model is load, there is a common interface to evaluate its performance, on a dataset. This is showed in the cell bellow where we evaluate a physics based simulator `DCApproximation` on these two dataset.

In [6]:
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
dc_metrics_per_dataset = benchmark1.evaluate_simulator(augmented_simulator=dc_sim,
                                                       dataset="all", # other values : "val", "test", "test_ood_topo"
                                                       save_path=EVAL_SAVE_PATH,
                                                       save_predictions=True
                                                      )

evaluate dc: 100%|█████████████████████████████████████████████████████| 10000/10000 [03:18<00:00, 50.36it/s]


### ML-related performances

#### Test dataset evaluation

In [228]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset["test"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset["test"]["ML"]["mape_90_avg"]["a_ex"])

MAPE90 for a_or:  0.1408511172541828
MAPE90 for a_ex:  0.13240412951825123


#### OOD Generalization evaluation

In [229]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset["test_ood_topo"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset["test_ood_topo"]["ML"]["mape_90_avg"]["a_ex"])

MAPE90 for a_or:  0.1608534769924919
MAPE90 for a_ex:  0.149238631980091


### Physics compliances

In [230]:
dc_metrics_per_dataset["test"]["Physics"].keys()

dict_keys(['CURRENT_POS'])

TODO : when there is no violation, create although the key and force it to take value zero

#### Test Dataset Evaluation

In [231]:
print("1) Current positivity violation:", dc_metrics_per_dataset["test"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]

1) Current positivity violation: {}


#### OOD Generalization Evaluation

In [232]:
print("1) Current positivity violation:", dc_metrics_per_dataset["test_ood_topo"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]

1) Current positivity violation: {}


And now it is possible to study the metrics on the different dataset. For example, if we want the "MSE" error on the "test" dataset (with a similar distribution as the training one):

## A learned baseline "augmented simulator"

Along with some dataset, we provide also some baseline (from a trained neural network). This baseline is made of a fully connected neural network that takes the available input of the powergrid and tries to predict all the output of the simulator.

The fully connected neural network is made of XXX layer each with YYY units.

It is learned for KKK epochs on the training set of the `Benchmark1`.

**NB** These baselines are not yet fully trained, and some hyper parameters still need to be optimized. We intend on doing that before the official release of the benchmark for the Neurips conference.

First we need to load the baseline and initialize it properly

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

# rebuild the baseline architecture
tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

Then, as for the DC approximation, we can evaluate it on the test datasets of the benchmark.

This is done with the same command:

In [18]:
# TODO: log the losses
# EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                              eval_batch_size=10000,
                                              dataset="all",
                                              shuffle=False,
                                              save_path=None,
                                              save_predictions=False
                                             )

2022-05-14 19:11:16.036827: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


## Comparison of the two augmented simulator

### Machine learning metrics 

And now we can compare the two "augmented simulators". For example, if we want to compare the MAPE90 (mean absolute percentage error compute for last 10% quantile) on the test dataset (with a distribution similar to the training distribution) for currents (A) at two extremity of power lines, we might compare:

In [19]:
ML_metrics = "ML"
dataset_name = "test"
print("DC Approximation")
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("MAPE90", dc_metrics_per_dataset[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MAE_avg"]))
dataset_name = "test_ood_topo"
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("mape_90_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", dc_metrics_per_dataset[dataset_name][ML_metrics]["MAE_avg"]))

DC Approximation
Dataset : test
MAPE90     : {'a_or': 0.2054102762347278, 'a_ex': 0.18943869874874716}
MSE_avg    : {'a_or': 91723.53006148033, 'a_ex': 131732.91079220167}
MAE_avg    : {'a_or': 114.3059446624192, 'a_ex': 143.16579138645758}
Dataset : test_ood_topo
mape_90_avg : {'a_or': 0.2227085624215881, 'a_ex': 0.20711185760666956}
MSE_avg    : {'a_or': 107658.54513807358, 'a_ex': 155649.85692985042}
MAE_avg    : {'a_or': 123.67567242785628, 'a_ex': 155.63737545351296}


In [21]:
ML_metrics = "ML"
dataset_name = "test"
print("Fully Connected Augmented Simulator")
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("mape_90_avg", tf_fc_metrics[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MAE_avg"]))
dataset_name = "test_ood_topo"
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("mape_90_avg", tf_fc_metrics[dataset_name][ML_metrics]["mape_90_avg"]))
print("{:<10} : {}".format("MSE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MSE_avg"]))
print("{:<10} : {}".format("MAE_avg", tf_fc_metrics[dataset_name][ML_metrics]["MAE_avg"]))

Fully Connected Augmented Simulator
Dataset : test
mape_90_avg : {'a_or': 0.005971284397153569, 'a_ex': 0.005959986222812791}
MSE_avg    : {'a_or': 21.65915870666504, 'a_ex': 44.23374557495117}
MAE_avg    : {'a_or': 2.7974419593811035, 'a_ex': 3.8893446922302246}
Dataset : test_ood_topo
mape_90_avg : {'a_or': 0.17957805279569528, 'a_ex': 0.17880120699511942}
MSE_avg    : {'a_or': 12364.6455078125, 'a_ex': 23608.12109375}
MAE_avg    : {'a_or': 52.66007614135742, 'a_ex': 74.55868530273438}


### Physic compliance

In [22]:
physic_compliances = "Physics"
dataset_name = "test"
current_violation = tf_fc_metrics[dataset_name][physic_compliances]["CURRENT_POS"]["a_or"]["Violation_proportion"]
print("{:.2f}% of currents at the origin side of power lines violate the current positivity.".format(current_violation*100))

2.23% of currents at the origin side of power lines violate the current positivity.


In [23]:
current_error = tf_fc_metrics[dataset_name][physic_compliances]["CURRENT_POS"]["a_or"]["Violation_proportion"]
print("The sum of negative current values (Amp) : {:.2f}".format(current_error))

The sum of negative current values (Amp) : 0.02


### Industrial readiness

In [24]:
tf_fc.predict_time

0.033661603927612305

In [25]:
dc_sim._predict_time

134.44738698005676

In [26]:
dc_sim._raw_grid_simulator.comp_time

76.7556830085814

# Benchmark2

In [3]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

2022-06-16 10:33:38.986746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 10:33:38.987008: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# to verify the config is loaded appropriately for this benchmark
print("Benchmark name: ", benchmark2.config.section_name)
print("Environment name: ", benchmark2.config.get_option("env_name"))
print("Output attributes: ", benchmark2.config.get_option("attr_y"))
print("Evaluation criteria: ")
pprint(benchmark2.config.get_option("eval_dict"))

Benchmark name:  Benchmark2
Environment name:  l2rpn_case14_sandbox
Output attributes:  ('a_or', 'a_ex', 'p_or', 'p_ex', 'v_or', 'v_ex')
Evaluation criteria: 
{'IndRed': ['TIME_INF'],
 'ML': ['MSE_avg', 'MAE_avg', 'mape_avg', 'mape_90_avg', 'TIME_INF'],
 'OOD': ['MSE_avg', 'MAE_avg', 'mape_avg', 'mape_90_avg', 'TIME_INF'],
 'Physics': ['CURRENT_POS',
             'VOLTAGE_POS',
             'LOSS_POS',
             'DISC_LINES',
             'CHECK_LOSS',
             'CHECK_GC',
             'CHECK_LC',
             'CHECK_VOLTAGE_EQ']}


In [5]:
# the next few lines are specific for each benchmark and each `AugmentedSimulator`
import grid2op
import warnings
from lips.physical_simulator.dcApproximationAS import DCApproximationAS
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    env = grid2op.make(benchmark2.config.get_option("env_name"), test=True)
    grid_path = pathlib.Path(env.get_path_env()) / "grid.json"

dc_sim = DCApproximationAS(name="dc_approximation", 
                           benchmark_name="Benchmark2",
                           config_path=BENCH_CONFIG_PATH,
                           grid_path=grid_path)

In [ ]:
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark2)
dc_metrics_per_dataset_bench2 = benchmark2.evaluate_simulator(augmented_simulator=dc_sim,
                                                              dataset="all", # other values : "val", "test", "test_ood_topo"
                                                              save_path=EVAL_SAVE_PATH,
                                                              save_predictions=True
                                                              )

### ML-related performances

#### Test dataset evaluation

In [20]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset_bench2["test"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset_bench2["test"]["ML"]["mape_90_avg"]["a_ex"])
print("MAPE for p_or: ", dc_metrics_per_dataset_bench2["test"]["ML"]["mape_avg"]["p_or"])
print("MAPE for p_ex: ", dc_metrics_per_dataset_bench2["test"]["ML"]["mape_avg"]["p_ex"])
print("MAE for p_or: ", dc_metrics_per_dataset_bench2["test"]["ML"]["MAE_avg"]["p_or"])
print("MAE for p_ex: ", dc_metrics_per_dataset_bench2["test"]["ML"]["MAE_avg"]["p_ex"])
print("MAPE for v_or: ", dc_metrics_per_dataset_bench2["test"]["ML"]["mape_avg"]["v_or"])
print("MAPE for v_ex: ", dc_metrics_per_dataset_bench2["test"]["ML"]["mape_avg"]["v_ex"])
print("MAE for v_or: ", dc_metrics_per_dataset_bench2["test"]["ML"]["MAE_avg"]["v_or"])
print("MAE for v_ex: ", dc_metrics_per_dataset_bench2["test"]["ML"]["MAE_avg"]["v_ex"])

MAPE90 for a_or:  0.1575609916428136
MAPE90 for a_ex:  0.14730832052856613
MAPE for p_or:  0.0921423999325571
MAPE for p_ex:  0.08787188867809301
MAE for p_or:  1.140607839753558
MAE for p_ex:  1.026569282669907
MAPE for v_or:  0.020920751203534725
MAPE for v_ex:  0.033166301338939726
MAE for v_or:  0.6651214754772188
MAE for v_ex:  0.9825881892442705


#### OOD Generalization evaluation

In [21]:
print("MAPE90 for a_or: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["mape_90_avg"]["a_or"])
print("MAPE90 for a_ex: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["mape_90_avg"]["a_ex"])
print("MAPE for p_or: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["mape_avg"]["p_or"])
print("MAPE for p_ex: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["mape_avg"]["p_ex"])
print("MAE for p_or: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["MAE_avg"]["p_or"])
print("MAE for p_ex: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["MAE_avg"]["p_ex"])
print("MAPE for v_or: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["mape_avg"]["v_or"])
print("MAPE for v_ex: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["mape_avg"]["v_ex"])
print("MAE for v_or: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["MAE_avg"]["v_or"])
print("MAE for v_ex: ", dc_metrics_per_dataset_bench2["test_ood_topo"]["ML"]["MAE_avg"]["v_ex"])

MAPE90 for a_or:  0.16061949592695715
MAPE90 for a_ex:  0.15209429645818998
MAPE for p_or:  0.08811211862205569
MAPE for p_ex:  0.08532031022804058
MAE for p_or:  1.1736193180608905
MAE for p_ex:  1.0713623671817072
MAPE for v_or:  0.021399578333099696
MAPE for v_ex:  0.03433434072231821
MAE for v_or:  0.7879805879878998
MAE for v_ex:  1.119801747493744


### Physics compliances

In [9]:
dc_metrics_per_dataset_bench2["test"]["Physics"].keys()

dict_keys(['CURRENT_POS', 'VOLTAGE_POS', 'LOSS_POS', 'DISC_LINES', 'CHECK_LOSS', 'CHECK_GC', 'CHECK_LC', 'CHECK_VOLTAGE_EQ'])

TODO : when there is no violation, create although the key and force it to take value zero

#### Test Dataset Evaluation

In [10]:
print("1) Current positivity violation:", dc_metrics_per_dataset_bench2["test"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", dc_metrics_per_dataset_bench2["test"]["Physics"]["VOLTAGE_POS"])
print("3) Loss positivity violation:", dc_metrics_per_dataset_bench2["test"]["Physics"]["LOSS_POS"])
print("4) Disconnected lines violation:", dc_metrics_per_dataset_bench2["test"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", dc_metrics_per_dataset_bench2["test"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_bench2["test"]["Physics"]["CHECK_GC"]["violation_percentage"], dc_metrics_per_dataset_bench2["test"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_bench2["test"]["Physics"]["CHECK_LC"]["violation_percentage"], dc_metrics_per_dataset_bench2["test"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", dc_metrics_per_dataset_bench2["test"]["Physics"]["CHECK_VOLTAGE_EQ"][0][2])

1) Current positivity violation: {}
2) Voltage positivity violation: {}
3) Loss positivity violation: {}
4) Disconnected lines violation: {}
5) Violation of loss to be between [1,4]% of production: 0.0
6) Violation of global conservation: 100.0% and its weighted mape: 0.9999998971074707
7) Violation of local conservation: 7.142857142857142% and its weighted mape: 0.01564590303591114
8) Violation proportion of voltage equality at subs: 0.5333333333333333


#### OOD Generalization Evaluation

In [11]:
print("1) Current positivity violation:", dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CURRENT_POS"])#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["VOLTAGE_POS"])
print("3) Loss positivity violation:", dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["LOSS_POS"])
print("4) Disconnected lines violation:", dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CHECK_GC"]["violation_percentage"], dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CHECK_LC"]["violation_percentage"], dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", dc_metrics_per_dataset_bench2["test_ood_topo"]["Physics"]["CHECK_VOLTAGE_EQ"][0][2])

1) Current positivity violation: {}
2) Voltage positivity violation: {}
3) Loss positivity violation: {}
4) Disconnected lines violation: {}
5) Violation of loss to be between [1,4]% of production: 0.0
6) Violation of global conservation: 100.0% and its weighted mape: 1.0000000047503754
7) Violation of local conservation: 7.142857142857142% and its weighted mape: 0.017686318493588492
8) Violation proportion of voltage equality at subs: 0.5


### Industrial Readiness

In [16]:
print(f'Inference time for test dataset: {dc_metrics_per_dataset_bench2["test"]["IndRed"]["TIME_INF"]:.2f}s')
print(f'Inference time for OOD dataset: {dc_metrics_per_dataset_bench2["test_ood_topo"]["IndRed"]["TIME_INF"]:.2f}s')

Inference time for test dataset: 82.65s
Inference time for OOD dataset: 83.23s


In [9]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

# rebuild the baseline architecture
tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark2",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

# LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
# tf_fc.restore(LOAD_PATH)

In [ ]:
tf_fc.train(train_dataset=benchmark2.train_dataset, val_dataset=benchmark2.val_dataset, epochs=20)

In [11]:
tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                              eval_batch_size=10000,
                                              dataset="all",
                                              shuffle=False,
                                              save_path=None,
                                              save_predictions=False
                                             )

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:37<00:00, 102.52it/s]


In [18]:
print("1) Current positivity violation:", (tf_fc_metrics["test_ood_topo"]["Physics"]["CURRENT_POS"]["a_or"]["Violation_proportion"]+tf_fc_metrics["test_ood_topo"]["Physics"]["CURRENT_POS"]["a_ex"]["Violation_proportion"])/2)#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", (tf_fc_metrics["test_ood_topo"]["Physics"]["VOLTAGE_POS"]["v_or"]["Violation_proportion"]+tf_fc_metrics["test_ood_topo"]["Physics"]["VOLTAGE_POS"]["v_ex"]["Violation_proportion"])/2)
print("3) Loss positivity violation:", tf_fc_metrics["test_ood_topo"]["Physics"]["LOSS_POS"]["violation_proportion"])
print("4) Disconnected lines violation:", tf_fc_metrics["test_ood_topo"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_GC"]["violation_percentage"], tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_LC"]["violation_percentage"], tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_VOLTAGE_EQ"][0][2])

1) Current positivity violation: 0.0145525
2) Voltage positivity violation: 0.01344
3) Loss positivity violation: 0.334525
4) Disconnected lines violation: {}
5) Violation of loss to be between [1,4]% of production: 2.04
6) Violation of global conservation: 99.97% and its weighted mape: 0.32163846492767334
7) Violation of local conservation: 98.97785714285715% and its weighted mape: 0.039185661898096955
8) Violation proportion of voltage equality at subs: 0.995625


In [19]:
tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_LC"]

{'lc_values': array([[ 0.93359566, -1.87155533, -1.41511726, ...,  0.4112649 ,
          0.44258046, -0.56777906],
        [ 0.41781998,  0.98659134,  1.32159805, ..., -0.2284466 ,
          0.37139964,  0.31681108],
        [-0.86855888, -3.11248207, -0.11334896, ...,  0.65899277,
         -0.02189493, -0.27778149],
        ...,
        [-0.87533188,  0.91049194,  0.57787037, ...,  0.12562943,
          0.46206784, -0.59373856],
        [ 1.54883957, -0.09885025, -0.13272953, ...,  0.11185193,
         -0.23499465, -0.79366875],
        [ 1.89224243,  0.84216881,  0.35978317, ...,  0.6466496 ,
          1.34231901, -0.70930004]]),
 'violation_percentage': 98.97785714285715,
 'mae': 0.8387612063547331,
 'mape': 0.039185661898096955}

In [25]:
import numpy as np
np.sum(tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_LC"]["lc_values"], axis=1)

array([ 3.41328454,  0.91084516, -1.91290545, ...,  0.33432364,
       -0.97183847,  0.43700492])

In [20]:
tf_fc_metrics["test_ood_topo"]["Physics"]["CHECK_GC"]

{'gc_values': array([ 3.4132667 ,  0.91083765, -1.9129095 , ...,  0.33430815,
        -0.97188854,  0.43698454], dtype=float32),
 'violation_percentage': 99.97,
 'mae': 1.7046846,
 'wmape': 0.32163846}